In [25]:
import glob
# This file should run basically insta fast. Bad coding for sure 
# There is a place where i sometimes comment out lin/rbf/sigmoid stuff
# Dont do that, it really helps for some subjects. 
# Anything. Leave only linear. For fast testing of best features
# But dont do that either. 

genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "09" # Which month and days result you want to visualize in table
month = 11

repetitionValue = 52
repetitionName =   "udrliplotnoAda5"   # "udrliplotnoAda3"  #  "udrlBC4CVTest" # 
#  "udrliplotnoAda3hyperparams" 
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.1
print(len(savedResults))
print(len(savedResults)/9)

18
2.0


In [26]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                
                
                # if "lin" in akernel:
                #     continue
                # if "sig" in akernel:
                #     continue
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
nameArray = np.array(list(nameDict))
topPerFeature = np.zeros([10, len(nameDict)])
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]
            
            comboName = list(nameDict.keys())[combinationList[res[0]][0]]
            nameIndex = np.where(nameArray == comboName)[0][0]
            if res[1] > topPerFeature[subNr, nameIndex]:
                topPerFeature[subNr, nameIndex] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 729)
(10, 729)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [27]:
x = 0

pd.set_option("display.max_columns", None)
print()
nameDictNrs = np.ones([1, len(nameDict)])
nameDictNrs[0,:] = np.arange(len(nameDict)) * 3
print(nameDictNrs.shape)
#table = pd.DataFrame(nameDictNrs , columns=nameDict.keys())
table = pd.DataFrame(topPerFeature, columns=nameDict.keys())
#display(table)
display(table)
# for names in nameDict.keys():
#     print(names, end="        ")
#     x +=1
#     #print(x)



(1, 243)


,fftData-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,fftData-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,fftData-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,fftData-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,fftData-fftData-welchData-welchDataBC--dataHRCV-BCcn3,fftData-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,fftData-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,fftData-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,fftData-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,welchData-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,welchData-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,welchData-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,welchData-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,welchData-fftData-welchData-welchDataBC--dataHRCV-BCcn3,welchData-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,welchData-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,welchData-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,welchData-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,dataHR-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,dataHR-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,dataHR-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,dataHR-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,dataHR-fftData-welchData-welchDataBC--dataHRCV-BCcn3,dataHR-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,dataHR-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,dataHR-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,dataHR-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,dataCorr1d-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,dataCorr1d-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,dataCorr1d-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,dataCorr1d-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,dataCorr1d-fftData-welchData-welchDataBC--dataHRCV-BCcn3,dataCorr1d-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,dataCorr1d-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,dataCorr1d-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,dataCorr1d-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,dataFFTCV-BC-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,dataFFTCV-BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,dataFFTCV-BC-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,dataFFTCV-BC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,dataFFTCV-BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3,dataFFTCV-BC-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,dataFFTCV-BC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,dataFFTCV-BC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,dataFFTCV-BC-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,dataWCV-BC-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,dataWCV-BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,dataWCV-BC-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,dataWCV-BC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,dataWCV-BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3,dataWCV-BC-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,dataWCV-BC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,dataWCV-BC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,dataWCV-BC-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,dataHRCV-BC-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,dataHRCV-BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,dataHRCV-BC-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,dataHRCV-BC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC,dataHRCV-BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3,dataHRCV-BC-welchDataBC-dataCorr1d-dataHRcn3--fftDatacn3BC,dataHRCV-BC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC,dataHRCV-BC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC,dataHRCV-BC-fftDatacn3-dataHRcn3-welchDataBC--welchDatacn3BC,fftDataBC-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3,fftDataBC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC,fftDataBC-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC,fftDataBC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--ff

In [28]:
import matplotlib.pyplot as plt
if len(nameArray) < 40:
    plt.figure(figsize=[30,10])
    plt.boxplot(topPerFeature)
    plt.legend(nameArray)
    print("")

In [29]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.84 : subject 2, feature:dataWCV-BCcn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernellinear, C:2.5
Accuracy 0.84 : subject 2, feature:dataWCV-BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernellinear, C:2.5
Accuracy 0.83 : subject 9, feature:welchDatacn3BC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC, kernellinear, C:2.5
Accuracy 0.83 : subject 9, feature:gaussianDatacn3BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3, kernellinear, C:2.5
Accuracy 0.83 : subject 9, feature:dataHRcn3BC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 2, feature:welchDatacn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 2, feature:dataHRCV-BCcn3-dataWCV-BC-dataGCVBC-welchDataBC--welchDatacn3, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 2, feature:dataHRCV-BCcn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernellinear, C:2.5
Accu

In [30]:
# top for each feature

In [31]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop[:1]:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.84 : subject 2, feature:dataWCV-BCcn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernellinear, C:2.5

Accuracy 0.83 : subject 9, feature:welchDatacn3BC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC, kernellinear, C:2.5

Accuracy 0.81 : subject 8, feature:gaussianDataBC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC, kernellinear, C:2.5

Accuracy 0.8 : subject 1, feature:gaussianDataBC-dataFFTCV-BC-dataCorr1d-fftDataBC--gaussianDatacn3BC, kernelsigmoid, C:2.5

Accuracy 0.79 : subject 5, feature:fftData-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernellinear, C:2.5

Accuracy 0.76 : subject 3, feature:fftDatacn3BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC, kernelsigmoid, C:2.5

Accuracy 0.76 : subject 4, feature:welchData-dataWCV-BC-welchData-fftDatacn3BC--welchDatacn3BC, kernellinear, C:2.5

Accuracy 0.75 : subject 7, feature:fftDatacn3BC-dataFFTCV-BC-fftDatacn3-fftDataBC--dataCorr1dcn3BC, kernelrbf, C:2.5

A

In [32]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.7044353  0.70358939 0.6896624  0.69497314 0.70227481
 0.68669722 0.6836134  0.71676955 0.7284808 ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728
0,0.696051,0.692866,0.68522,0.728458,0.715011,0.721724,0.721009,0.699684,0.704237,0.720903,0.701564,0.691835,0.714569,0.700554,0.714646,0.700835,0.720518,0.697924,0.69039,0.684603,0.687682,0.701326,0.69643,0.687661,0.700196,0.718357,0.687051,0.674263,0.699011,0.67681,0.716007,0.709224,0.713643,0.716968,0.697033,0.703774,0.715039,0.706117,0.688258,0.709708,0.699502,0.71781,0.707471,0.715965,0.695609,0.696023,0.681362,0.683439,0.700372,0.696254,0.695455,0.701838,0.708403,0.693378,0.698997,0.700035,0.672924,0.710038,0.704209,0.713335,0.714268,0.699348,0.700351,0.706629,0.696395,0.68494,0.713279,0.70611,0.717579,0.703409,0.707884,0.695455,0.698772,0.679195,0.689843,0.7121,0.711378,0.690902,0.694613,0.711567,0.690678,0.68555,0.707807,0.681229,0.72265,0.70935,0.707934,0.714043,0.69549,0.698443,0.71095,0.696472,0.691835,0.713335,0.697804,0.707962,0.705542,0.710795,0.698387,0.692424,0.674621,0.685753,0.703304,0.70209,0.688559,0.70719,0.709947,0.692424,0.704896,0.701403,0.684757,0.728381,0.712079,0.710585,0.718126,0.699453,0.701122,0.7172,0.699944,0.693098,0.719606,0.708067,0.719999,0.707471,0.718512,0.69584,0.695251,0.678402,0.682667,0.709428,0.702813,0.692214,0.702869,0.710332,0.68

Max average accuracy 0.7369037598204264


In [33]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

fftDatacn3BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3
linear
0.7369037598204264

welchDataBC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC
linear
0.7357182940516274

fftDatacn3BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
linear
0.732168911335578

fftDatacn3BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
sigmoid
0.7315516273849607

dataFFTCV-BCcn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
linear
0.7309273288439955



In [34]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboTotal.argsort()[0][-5:][::-1]]
top5Vals = comboTotal[0][comboTotal.argsort()[0][-5:][::-1]] /9
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

fftDatacn3BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3
linear
0.7369037598204264

welchDataBC-fftDatacn3-dataHRCV-BCcn3-dataCorr1d--fftDatacn3BC
linear
0.7357182940516274

fftDatacn3BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
linear
0.732168911335578

fftDatacn3BC-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
sigmoid
0.7315516273849607

dataFFTCV-BCcn3-dataGCVBC-dataWCV-BCcn3-dataHRCV-BCcn3--welchDataBC
linear
0.7309273288439955



In [35]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728
0,6.264457,6.235795,6.166982,6.556124,6.435101,6.495518,6.489078,6.297159,6.338131,6.488131,6.314078,6.226515,6.431124,6.304987,6.431818,6.307513,6.484659,6.281313,6.21351,6.161427,6.189141,6.311932,6.267866,6.188952,6.301768,6.465215,6.18346,6.068371,6.291098,6.091288,6.444066,6.383018,6.42279,6.452715,6.273295,6.333965,6.435354,6.355051,6.194318,6.387374,6.295518,6.46029,6.367235,6.443687,6.26048,6.264205,6.13226,6.150947,6.303346,6.266288,6.259091,6.31654,6.375631,6.240404,6.290972,6.300316,6.056313,6.390341,6.337879,6.420013,6.428409,6.294129,6.303157,6.359659,6.267551,6.164457,6.419508,6.354987,6.458207,6.330682,6.37096,6.259091,6.288952,6.112753,6.208586,6.408902,6.402399,6.218119,6.251515,6.404104,6.216098,6.169949,6.370265,6.131061,6.503851,6.384154,6.371402,6.426389,6.259407,6.285985,6.398548,6.268245,6.226515,6.420013,6.28024,6.371654,6.349874,6.397159,6.28548,6.231818,6.071591,6.17178,6.329735,6.318813,6.197033,6.36471,6.38952,6.231818,6.344066,6.312626,6.162816,6.555429,6.408712,6.395265,6.463131,6.295076,6.310101,6.454798,6.299495,6.237879,6.476452,6.372601,6.479987,6.367235,6.466604,6.262563,6.25726,6.105619,6.144003,6.384848,6.325316,6.229924,6.325821,6.392992

579
9.0
Max total significant accuracy then averaged 0.7369037598204264 
fftDatacn3BC-fftData-welchData-welchDataBC--dataHRCV-BCcn3
linear
C: 2.5 


In [36]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
